#Step by step design for the two resonator single xmon design. 


## Sections
### I. Transmon only
1. Prepare the layout in qiskit-metal. <br>
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Set up EPR junction dictionary. <br>
1. Run EPR analysis on single mode. <br>
1. Get qubit freq and anharmonicity. <br>
1. Calculate EPR of substrate.  <br>
1. (Extra: Calculate surface EPR.) <br>

### II. Resonator only
1. Update the layout in qiskit-metal. <br>
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Calculate EPR of substrate. <br>
    
### III. Transmon & resonator
1. Update the layout in qiskit-metal. <br>
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Set up EPR junction dictionary. <br>
1. Run EPR analysis on the two modes. <br>
1. Get qubit frequency and anharmonicity. <br>

### IV. Transmon & resonator
1. Update the layout in qiskit-metal. <br>
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Set up EPR junction dictionary. <br>
1. Run EPR analysis on the two modes. <br>
1. Get qubit frequency and anharmonicity. <br>

### V. Analyze the coupled  transmon 2 resonator system. 
1. Finite Element Eigenmode Analysis
1. Identify the mode you want. The mode can inclusively be from 1 to setup.n_modes.
1. Set variables in the Ansys design. As before, we seek 2 modes.
1. Set up the simulation and specify the variables for the sweep.
1. Plot the E-field on the chip's surface.
1. Specify the junctions in the model; in this case there are 2 junctions.
1. Find the electric and magnetic energy stored in the substrate and the system as a whole.
1. Perform EPR analysis for all modes and variations.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path
sys.path.remove('C:\\Users\\wanglab2\\Documents\\pyEPR')

sys.path


['C:\\Users\\wanglab2\\github\\qiskit-metal\\tutorials\\4 Analysis\\A. Core - EM and quantization2\\A. Core - EM and quantization',
 'C:\\Users\\wanglab2\\Anaconda3\\envs\\py310_lolo\\python310.zip',
 'C:\\Users\\wanglab2\\Anaconda3\\envs\\py310_lolo\\DLLs',
 'C:\\Users\\wanglab2\\Anaconda3\\envs\\py310_lolo\\lib',
 'C:\\Users\\wanglab2\\Anaconda3\\envs\\py310_lolo',
 '',
 'C:\\Users\\wanglab2\\Anaconda3\\envs\\py310_lolo\\lib\\site-packages',
 'c:\\users\\wanglab2\\github\\qiskit-metal',
 'C:\\Users\\wanglab2\\Anaconda3\\envs\\py310_lolo\\lib\\site-packages\\win32',
 'C:\\Users\\wanglab2\\Anaconda3\\envs\\py310_lolo\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\wanglab2\\Anaconda3\\envs\\py310_lolo\\lib\\site-packages\\Pythonwin']

In [2]:

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr
%metal_heading Qiskit Metal works fine! - Here are the design components!

In [3]:
from qiskit_metal.qlibrary.qubits.transmon_pocket_cl import TransmonPocketCL
from qiskit_metal.qlibrary.qubits.transmon_cross_fl import TransmonCrossFL
from qiskit_metal.qlibrary.qubits.transmon_cross_fl_cl import TransmonCrossFL_CL
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
from qiskit_metal.qlibrary.couplers.tunable_coupler_01 import TunableCoupler01

from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround

from qiskit_metal.qlibrary.lumped.cap_n_interdigital import CapNInterdigital
from qiskit_metal.qlibrary.lumped.cap_3_interdigital import Cap3Interdigital
from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.launchpad_wb_simplecoupler import LaunchpadWirebondSimple
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled

from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
import numpy as np
from collections import OrderedDict


In [4]:
design = metal.designs.DesignPlanar()

design.chips.main.size['size_x'] = '5mm'
design.chips.main.size['size_y'] = '5mm'
design.chips.main.size['size_z'] = '-725um'
gui = MetalGUI(design)

# Resonator length estimate

In [5]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"


find_resonator_length(6.935, 10, 6, 4)

'4.3881350926847285 mm'

# Xmon qubits 

In [6]:
design.overwrite_enabled = True
options =  dict(
    connection_pads=dict(
        readout = dict(loc_W=+1, loc_H=+1),
        storage = dict(loc_W=-1, loc_H=-1, )
    ))
    
from qiskit_metal.qlibrary.qubits.transmon_cross_fl import TransmonCrossFL
from qiskit_metal.qlibrary.qubits.transmon_cross_fl_cl import TransmonCrossFL_CL

Q1 = TransmonCrossFL_CL(design, 'Q1', options = dict(make_extra_rect=True, pos_x = '-0.2mm', pos_y='-1.450mm', 
                                                     cross_length='180um', cross_width='30um', cross_gap='30um',
                                                    # cross_gap='3um',
                                                 connection_pads = dict(
                                                     storage = dict(connector_location = '180',
                                                      claw_length='140um', claw_width='15um', ground_spacing='4um', 
                                                                    claw_gap='10um', lead_width='25um',  lead_length='40um',
                                                      lead_gap='15um' ),  #,claw_length ='115um', claw_width='25um', claw_gap='7um', 
                                                     #ground_spacing='3um'),
                                                     readout = dict(connector_location = '0', lead_gap='6um',
                                                     claw_length='112um', claw_width='15um', ground_spacing='4um', 
                                                                    claw_gap='10um', lead_width='10um', lead_length='60um')),#claw_length ='115um', claw_width='25um', claw_gap='7um',ground_spacing='3um')),
                                                 fl_options = dict(t_shift='3.5um' , t_offset='-6um', t_inductive_gap='4.5um', 
                                                                   t_top='17.5um', t_width='5um', t_gap='3um'),
                                                 cl_options= dict(c_offset='2um', c_ground_gap='38um', c_width='5um', 
                                                                  c_gap='3um', translate_x='0um'),
                                                 rect_options= dict(rect_width='40um', rect_length='8um')))
    


Q2 = TransmonCrossFL_CL(design, 'Q2', options = dict(make_extra_rect=True, pos_x = '-0.2mm', pos_y='1.450mm', orientation=0, 
                                                     cross_length='187um', cross_width='30um', cross_gap='30um',
                                                    # cross_gap='3um',
                                                 connection_pads = dict(
                                                     storage = dict(connector_location = '180',
                                                      claw_length='150um', claw_width='15um', ground_spacing='4um', 
                                                                    claw_gap='10um', lead_width='35um',  lead_length='40um',
                                                      lead_gap='21um' ),  #,claw_length ='115um', claw_width='25um', claw_gap='7um', 
                                                     #ground_spacing='3um'),
                                                     readout = dict(connector_location = '0', lead_gap='6um',
                                                     claw_length='110um', claw_width='15um', ground_spacing='4um', 
                                                                    claw_gap='10um', lead_width='10um', lead_length='60um')),#claw_length ='115um', claw_width='25um', claw_gap='7um',ground_spacing='3um')),
                                                 fl_options = dict(t_shift='3.5um' , t_offset='-6um', t_inductive_gap='4.5um', 
                                                                   t_top='17.5um', t_width='5um', t_gap='3um'),
                                                 cl_options= dict(c_offset='2um', c_ground_gap='38um', c_width='5um', 
                                                                  c_gap='3um', translate_x='0um'),
                                                 rect_options= dict(rect_width='40um', rect_length='8um')))

gui.rebuild()
gui.autoscale()

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)



# Flux Lines and Flux Launchpads

In [7]:

import numpy as np
from collections import OrderedDict


anchors1f = OrderedDict()
anchors1f[0] = np.array([-1.75, -2.0])



launch_flux_options = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                           pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um', 
                           pos_x='-1.954mm', pos_y='-1.8mm', orientation='0', lead_length='30um')
lf = LaunchpadWirebond(design, 'Launch_Flux', options = launch_flux_options)


flux_line_options = Dict(
        lead = dict(start_straight='100um', end_straight='100um'),
        fillet='30um', trace_width = '5um', trace_gap = '3um', 
        pin_inputs=Dict(
            start_pin=Dict(component='Q1', pin='flux_line'),
            end_pin=Dict(componentl=lf.name, pin='tie')), )
flux_line_Q1 = RouteAnchors(design,'Flux_Line_1', options = dict(hfss_wire_bonds = True,
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q1',
                                                    pin='flux_line'),
                                                end_pin=Dict(
                                                    component='Launch_Flux',
                                                    pin='tie')),
                                            lead=Dict(
#                                                 start_straight='175um',
                                                 end_straight = '100um',
                                                #end_jogged_extension = jogs_end
                                            ),
                                                    fillet = '60um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                             anchors = anchors1f
                                            ))




anchors2fl = OrderedDict()
anchors2fl[0] = np.array([1.78, 0.415])



launch_flux2_options = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                             pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um',  pos_x='1.954', pos_y='0.60081mm', orientation='180', lead_length='30um')
lf2 = LaunchpadWirebond(design, 'Launch_Flux2', options = launch_flux2_options)




flux_line_Q2 = RoutePathfinder(design,'Flux_Line_2', options = dict(hfss_wire_bonds = True,
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q2',
                                                    pin='flux_line'),
                                                end_pin=Dict(
                                                    component='Launch_Flux2',
                                                    pin='tie')),
                                            lead=Dict(
#                                                 start_straight='175um',
                                                 end_straight = '100um',
                                                #end_jogged_extension = jogs_end
                                            ),
                                            fillet = '60um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                            anchors=anchors2fl
                                            ))


gui.rebuild()

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)



# Charge Lines and Charge Launchpads

In [8]:






launch_charge_options = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                             pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um',  pos_x='1.954mm', pos_y='-0.59919mm', orientation='180', lead_length='30um')

lc = LaunchpadWirebond(design, 'Launch_Charge', options = launch_charge_options)






launch_charge2_options = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                             pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um',  pos_x='-1.954mm', pos_y='1.8mm', orientation='0', lead_length='30um')

lc2 = LaunchpadWirebond(design, 'Launch_Charge2', options = launch_charge2_options)


anchors1ch = OrderedDict()
anchors1ch[0] = np.array([1.78, 0.114])


charge_line_Q1 = RoutePathfinder(design,'Charge_Line_1', options = dict(hfss_wire_bonds = True,
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q1',
                                                    pin='charge_line'),
                                                end_pin=Dict(
                                                    component='Launch_Charge',
                                                    pin='tie')),
                                            lead=Dict(
                                                start_straight='175um',
                                                end_straight = '100um',
                                                #end_jogged_extension = jogs_end
                                            ),
                                            fillet = '60um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                            anchors = anchors1ch
                                            ))

anchors2ch = OrderedDict()
anchors2ch[0] = np.array([-1.75,2.0])
charge_line_Q2 = RoutePathfinder(design,'Charge_Line_2', options = dict(hfss_wire_bonds = True,
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q2',
                                                    pin='charge_line'),
                                                end_pin=Dict(
                                                    component='Launch_Charge2',
                                                    pin='tie')),
                                            lead=Dict(
#                                                 start_straight='175um',
                                                 end_straight = '100um',
                                                #end_jogged_extension = jogs_end
                                            ),
                                            fillet = '60um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                            anchors = anchors2ch
                                            ))

gui.rebuild()



  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_arr

# Readout Tee's and Readout Resonators

In [9]:

TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='-1.27mm',
                                             pos_y='-0.6mm',
                                              coupling_length='400um', coupling_space='4um', open_termination=False))
TQ2 = CoupledLineTee(design, 'TQ2', options=dict(pos_x='-1.35mm',
                                             pos_y='0.6mm',
                                             coupling_length='410um', coupling_space='4um', orientation=180, 
                                                 open_termination=False))



options1 = Dict(
    total_length='3.458mm',
    hfss_wire_bonds = True,
    fillet='40um',
    lead = dict(start_straight='100um', end_straight='400um'),
    pin_inputs=Dict(
        start_pin=Dict(
            component='TQ1',
            pin='second_end'),
        end_pin=Dict(
            component='Q1',
            pin='readout')),
    meander=Dict(spacing='200um',
            asymmetry='0um'))
read_q1 = RouteMeander(design, 'read_q1', options=options1)




options2 = Dict(
     total_length='3.58mm',
     hfss_wire_bonds = True,
     fillet='40um',
     lead = dict(start_straight='650um'),
     pin_inputs=Dict(
         end_pin=Dict(
             component='TQ2',
             pin='second_end'),
         start_pin=Dict(
             component='Q2',
             pin='readout')),
     meander=Dict(spacing='250um',
             asymmetry='0um'))
read_q2 = RouteMeander(design, 'read_q2', options=options2)



gui.rebuild()
gui.autoscale()


  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_arr

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)



# Feedline and Feedline Launchpads 

In [10]:
launch_feed_left_options = dict(trace_width= '10um',pad_width='150um', pad_gap_x= '96um', pad_gap_y= '130um', pad_height='200um', 
                           taper_height='150um',  pos_x='-1.954mm', pos_y='-0.6mm', orientation='0', lead_length='30um')
l_feed_left = LaunchpadWirebond(design, 'Launch_Feed_Left', options = launch_feed_left_options)


launch_feed_right_options = dict(trace_width= '10um', pad_width='150um', pad_gap_x= '96um', pad_gap_y= '130um', pad_height='200um', 
                           taper_height='150um',  pos_x='-1.954mm', pos_y='0.6mm', lead_length='30um', orientation='0')
l_feed_right = LaunchpadWirebond(design, 'Launch_Feed_Right', options = launch_feed_right_options)




anchors1c = OrderedDict()
anchors1c[0] = np.array([-0.65, 0.56079])



ops_oF = {'hfss_wire_bonds': 'True',
            'pin_inputs': 
            {'start_pin': {'component': 'TQ1', 'pin': 'prime_start'}, 
             'end_pin': {'component': 'Launch_Feed_Left', 'pin': 'tie'}},
            'fillet':'80um'
            }
cpw_openFirst = RouteStraight(design, 'cpw_openFirst', options=ops_oF)




ops_oM = {'hfss_wire_bonds': 'False',
            'pin_inputs': 
            {'start_pin': {'component': 'TQ1', 'pin': 'prime_end'}, 
             'end_pin': {'component': 'TQ2', 'pin': 'prime_start'}},
            'anchors': anchors1c,
            'fillet':'50um'
            }
cpw_openMid = RouteAnchors(design, 'cpw_openMid', options=ops_oM)





ops_oLast = {'hfss_wire_bonds': 'True',
            'pin_inputs': 
            {'start_pin': {'component': 'TQ2', 'pin': 'prime_end'}, 
             'end_pin': {'component': l_feed_right.name, 'pin': 'tie'}},
            'fillet':'80um'
            }
cpw_openLast = RouteStraight(design, 'cpw_openLast', options=ops_oLast)



gui.rebuild()
gui.autoscale()

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_arr

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)



# Storage Resonator 1, its drive line and launchpad

In [11]:
launch_storage_options = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                             pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um',  pos_x='1.954mm', pos_y='-1.8mm', orientation='180', lead_length='30um')
lc = LaunchpadWirebond(design, 'Launch_Storage', options = launch_storage_options)




otg11 = ShortToGround(design, 'otg11', options = dict(pos_x='1.231mm', 
                                                   pos_y='-1.54mm', orientation='180', termination_gap='3um', gap='3um', 
                                                     width='5um' ))
stg11= ShortToGround(design, 'stg11', options = dict(pos_x='1.314', pos_y='-1.283', orientation='90'))
                       
                       
jogs_start = OrderedDict()
jogs_start[0] = ["L", '1.225mm']                       
jogs_start[1] = ["R", '250um']  
jogs_start[2] = ["R", '1.225mm']                       
jogs_start[3] = ["L", '250um'] 
jogs_start[4] = ["L", '1.225mm']                       
jogs_start[5] = ["R", '250um'] 
jogs_start[6] = ["R", '1.225mm'] 
jogs_start[7] = ["L", '250um']
storage_trace_options1 = Dict(
        total_length='6.37mm', trace_width='25um', trace_gap='15um', 
        fillet='60 um',
        lead = dict(start_straight='250um', start_jogged_extension=jogs_start),
        hfss_wire_bonds = False,
        pin_inputs=Dict(
            
            start_pin=Dict(component='Q1', pin='storage'),
            end_pin=Dict(component='stg11', pin='short')),
            meander=Dict(spacing='405um',
            asymmetry='0um'))
store_q1_1 = RouteMeander(design, 'Store_Q1_1',  options=storage_trace_options1)



anchors1st = OrderedDict()
anchors1st[0] = np.array([1.78, -2])
storage_line_Q1 = RoutePathfinder(design,'Storage_Line_1', options = dict(hfss_wire_bonds = True,
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='otg11',
                                                    pin='short'),
                                                end_pin=Dict(
                                                    component='Launch_Storage',
                                                    pin='tie')),
                                            lead=Dict(end_straight='200um'),#, end_straight='100um'),
                                            fillet = '50um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                            #anchors=anchors1st
                                            ))



gui.rebuild()
gui.autoscale()

launch_storage_options2 = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                             pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um',  pos_x='1.954mm', pos_y='1.80081mm', orientation='180', lead_length='30um')
lc2 = LaunchpadWirebond(design, 'Launch_Storage2', options = launch_storage_options2)





04:38PM 12s INFO [connect_meandered]: Zero meanders for Store_Q1_1
  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

04:38PM 15s INFO [connect_meandered]: Zero meanders for Store_Q1_1
  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)



# Storage Resonator 2, its drive line and launchpad

In [12]:






launch_storage_options2 = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                             pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um',  pos_x='1.954mm', pos_y='1.80081mm', orientation='180', lead_length='30um')
lc2 = LaunchpadWirebond(design, 'Launch_Storage2', options = launch_storage_options2)


                       


otg11_2 = ShortToGround(design, 'otg11_2', options = dict(pos_x='1.357mm', 
                                                   pos_y='1.674mm', orientation='270'))#,termination_gap='3um', gap='3um', 
                                                     #width='5um'))
stg11_2= ShortToGround(design, 'stg11_2', options = dict(pos_x='1.271mm', pos_y='1.773mm', orientation='90'))
                       
                       
    

jogs_start = OrderedDict()
jogs_start[0] = ["L", '-0.750mm']                      
jogs_start[1] = ["R", '200um']  
jogs_start[2] = ["R", '-1mm']                       
jogs_start[3] = ["L", '200um'] 
jogs_start[4] = ["L", '-1mm']                       
jogs_start[5] = ["R", '200um'] 
jogs_start[6] = ["R", '-1mm'] 
jogs_start[7] = ["L", '200um']
jogs_start[8] = ["L", '-1mm']
jogs_start[9] = ["R", '200um']
jogs_start[10] = ["R", '-921um']
storage_trace_options2 = Dict(
        total_length='5.280mm', trace_width='35um', trace_gap='21um', 
        fillet='99 um',
        lead = dict(start_straight='200um', ends_straight='130um', start_jogged_extension=jogs_start),
        hfss_wire_bonds = False,
        pin_inputs=Dict(
            
            start_pin=Dict(component='Q2', pin='storage'),
            end_pin=Dict(component='stg11_2', pin='short')),)
#             meander=Dict(spacing='405um',
#             asymmetry='0um'))    
    
    
store_q2_2 = RouteMeander(design, 'Store_Q2_2',  options=storage_trace_options2)    
anchors2st = OrderedDict()


anchors2st[0] = np.array([1.774, 1.14])
# anchors2st[1] = np.array([-0.7, 2.27])
# anchors2st[2] = np.array([-0.7, 1.875])
    
storage_line_Q2 = RoutePathfinder(design,'Storage_Line_2', options = dict(hfss_wire_bonds = True,
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='otg11_2',
                                                    pin='short'),
                                                end_pin=Dict(
                                                    component='Launch_Storage2',
                                                    pin='tie')),
                                            lead=Dict(end_straight='150um'),
                                            fillet = '50um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                            #anchors=anchors2st
                                            ))
    
    
    
    


gui.rebuild()
gui.autoscale()

04:38PM 16s INFO [connect_meandered]: Zero meanders for Store_Q2_2
  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

04:38PM 19s INFO [connect_meandered]: Zero meanders for Store_Q1_1
  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

04:38PM 20s INFO [connect_meandered]: Zero meanders for Store_Q2_2
  arr = construct_1d_object_array_from_listlike(values)

  arr = con

In [13]:
%metal_heading Now analyzing the design: HFSS & EPR

# Running for an individual XMon

In [ ]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_qb = EPRanalysis(design, "hfss")

eig_qb = EPRanalysis(design, "hfss")

eig_qb.sim.setup.vars.Lj = '7.5 nH'
eig_qb.sim.setup.vars = Dict({'Lj': '7.5 nH', 'Cj': '3 fF'})
gui.rebuild()
gui.autoscale()

In [ ]:
design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q2'].options['hfss_inductance'] = 'Lj'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj'

gui.rebuild()
gui.autoscale()



In [ ]:
eig_qb.sim.setup

In [ ]:
eig_qb.sim.setup.max_passes = 15
eig_qb.sim.setup.vars.Lj = '7.5 nH'
eig_qb.sim.setup.vars = Dict({'Lj': '7.5 nH', 'Cj': '3 fF'})

eig_qb.sim.setup_update(max_delta_f = 0.5, min_freq_ghz = 3)
eig_qb.sim.setup.min_passes = 10
eig_qb.sim.setup.n_modes = 1

gui.rebuild()
gui.autoscale()



In [ ]:
eig_qb.sim.renderer._options['max_mesh_length_jj'] = '3um'
eig_qb.sim.renderer._options['max_mesh_length_port'] = '3um'

In [ ]:

design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q2'].options['hfss_inductance'] = 'Lj'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj'

gui.rebuild()
gui.autoscale()



eig_qb.sim.run(name="Xmon_only_new", components=['Q2'], open_terminations=[], box_plus_buffer = False)
eig_qb.sim.plot_convergences()

The last variables you pass to the `run()` or `sim.run()` methods, will be stored in the `sim.setup` dictionary under the key `run`. You can recall the information passed by either accessing the dictionary directly, or by using the print handle below.

In [ ]:
eig_qb.sim.plot_convergences()

In [ ]:
# eig_qb.setup.run    <- direct access
eig_qb.sim.print_run_args()

In [ ]:
eig_qb.setup.junctions.jj.rect = 'JJ_rect_Lj_Q1_rect_jj'
eig_qb.setup.junctions.jj.line = 'JJ_Lj_Q1_rect_jj_'
eig_qb.run_epr()

Checking the magnetic field of the modes.

In [ ]:
eig_qb.sim.plot_fields('main')  

hfss = eig_qb.sim.renderer
hfss.clear_fields(['Mag_E1'])
hfss.set_mode(1, "Setup")
hfss.modeler._modeler.ShowWindow()
hfss.plot_fields('main')
hfss.save_screenshot()

In [ ]:
eig_qb.sim.clear_fields()

# Xmon and resonators together

In [14]:
from qiskit_metal.analyses.quantization import EPRanalysis

In [15]:
design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q1'].options['hfss_inductance'] = 'Lj'
qcomps['Q1'].options['hfss_capacitance'] = 'Cj'
gui.rebuild()
gui.autoscale()

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_arr

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

10:57PM 59s INFO [connect_meandered]: Zero meanders for Store_Q1_1
  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

10:58PM 00s INFO [connect_meandered]: Zero meanders for Store_Q2_2
  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)



In [16]:
eig_rd = EPRanalysis(design, "hfss")

eig_rd.sim.setup.vars.Lj = '7.5 nH'
eig_rd.sim.setup.vars = Dict({'Lj': '7.5 nH', 'Cj': '3 fF'})
gui.rebuild()
gui.autoscale()

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_arr

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

10:58PM 03s INFO [connect_meandered]: Zero meanders for Store_Q1_1
  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

10:58PM 04s INFO [connect_meandered]: Zero meanders for Store_Q2_2
  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)



# Qubit 1 related simulations

In [19]:
eig_rd.sim.setup.n_modes =3

# eig_rd.sim.setup.min_passes = 1
# eig_rd.sim.setup.max_passes = 1

eig_rd.sim.setup.min_passes = 12
eig_rd.sim.setup.max_passes = 17


eig_rd.sim.setup_update(max_delta_f = 0.5, min_freq_ghz = 1.1)


design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q2'].options['hfss_inductance'] = 'Lj'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj'


#eig_rd.sim.run(name="xmon_and_readout_storage_final", components=['Q1', 'read_q1', 'Store_Q1_1', 'TQ1'], open_terminations=[], box_plus_buffer = False)
#eig_rd.sim.run(name="xmon_and_readout_87", components=['Q1', 'read_q1', 'TQ1'], open_terminations=[('TQ1', 'prime_start'), ('TQ1', 'prime_end')], box_plus_buffer = False)


#eig_rd.sim.run(name="readout4", components=['read_q1', 'TQ1', 'Q1'], open_terminations=[('TQ1', 'prime_end'), ('TQ1', 'prime_start')], box_plus_buffer = False)
eig_rd.sim.run(name="storage_and_xmon_and_readout_final3", components=['Q2', 'Store_Q2_2', 'read_q2', 'TQ2', 'TQ1', 'Launch_Feed_Left', 'Launch_Feed_Right', 'cpw_openFirst', 'cpw_openMid', 'cpw_openLast'], open_terminations=[('TQ1', 'prime_end'), ('TQ1', 'prime_start')], box_plus_buffer = False)
#eig_rd.sim.run(name="xmon_chargeline", components=['Q1', 'Charge_Line_1'], open_terminations=[('Charge_Line_1','end')])
# eig_rd.sim.run(name="open_24_131", components=['Q1', 'Store_Q1_1', 'Launch_Storage'], open_terminations=[('Launch_Storage','tie')], box_plus_buffer = False)
#eig_rd.sim.run(name="storage_coupling_PART96", components=['Q1','Store_Q1_1', 'Storage_Line_1', 'otg11', 'stg11'], open_terminations=[('Storage_Line_1', 'end')], box_plus_buffer = False)
#eig_rd.sim.run(name="ro_111622", components=['Q2', 'read_q2', 'TQ2', 'Store_Q2_2'], open_terminations=[], box_plus_buffer = False)
#eig_rd.sim.run(name="full_set2", components=['Q2', 'Store_Q2_2' 'read_q2', 'TQ2'], open_terminations=[], box_plus_buffer = False)

#eig_rd.sim.run(name="xmon_and_ro_set1", components=['Q1', 'Store_Q1_1', 'read_q1', 'TQ1'], open_terminations=[], box_plus_buffer = False)
#eig_rd.sim.run(name="storage_loss_fluxpad2", components=['Q2', 'Store_Q2_2', 'Launch_Flux2'], open_terminations=[], box_plus_buffer = False)
eig_rd.sim.plot_convergences()

INFO 10:59PM [connect_design]: 	Opened active design
	Design:    storage_and_xmon_and_readout_final3_hfss [Solution type: Eigenmode]
WARNING 10:59PM [connect_setup]: 	No design setup detected.
WARNING 10:59PM [connect_setup]: 	Creating eigenmode default setup.
INFO 10:59PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 11:00PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 11:00PM [analyze]: Analyzing setup Setup
11:05PM 50s INFO [get_f_convergence]: Saved convergences to C:\Users\wanglab2\github\qiskit-metal\tutorials\4 Analysis\A. Core - EM and quantization2\A. Core - EM and quantization\hfss_eig_f_convergence.csv


# Qubit 2 related simulations

In [17]:
eig_rd.sim.setup.n_modes =1

eig_rd.sim.setup.min_passes = 1
eig_rd.sim.setup.max_passes = 1

# eig_rd.sim.setup.min_passes = 11
# eig_rd.sim.setup.max_passes = 17


eig_rd.sim.setup_update(max_delta_f = 0.5, min_freq_ghz = 1.1)


design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q2'].options['hfss_inductance'] = 'Lj'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj'

gui.rebuild()
gui.autoscale()






#eig_rd.sim.run(name="storage_coupling_PART92", components=['Q2','Store_Q2_2', 'Storage_Line_2', 'otg11_2', 'stg11_2'], open_terminations=[('Storage_Line_2', 'end')], box_plus_buffer = False)




#eig_rd.sim.run(name="xmon_and_readout_storage_ayoy_39", components=['Q2', 'read_q2', 'Store_Q2_2', 'TQ2'], open_terminations=[], box_plus_buffer = False)
eig_rd.sim.run(name="xmon_and_readout_PART90", components=['Q2', 'read_q2', 'TQ2'], open_terminations=[('TQ2', 'prime_start'), ('TQ2', 'prime_end')], box_plus_buffer = False)


#eig_rd.sim.run(name="readout4", components=['read_q1', 'TQ1', 'Q1'], open_terminations=[('TQ1', 'prime_end'), ('TQ1', 'prime_start')], box_plus_buffer = False)
#eig_rd.sim.run(name="storage_and_xmon_and_readout", components=['Q1', 'Store_Q1', 'read_q1', 'TQ1', 'Launch_Feed_Left', 'Launch_Feed_Right', 'cpw_openRight', 'cpw_openLeft'], open_terminations=[], box_plus_buffer = False)
#eig_rd.sim.run(name="xmon_chargeline", components=['Q1', 'Charge_Line_1'], open_terminations=[('Charge_Line_1','end')])
# eig_rd.sim.run(name="open_24_131", components=['Q1', 'Store_Q1_1', 'Launch_Storage'], open_terminations=[('Launch_Storage','tie')], box_plus_buffer = False)
#eig_rd.sim.run(name="storage_coupling_ayoy", components=['Q1', 'Store_Q1_1', 'stg11', 'Storage_Line_1', 'otg11'], open_terminations=[('otg11', 'open'), ('Storage_Line_1', 'end')], box_plus_buffer = False)
#eig_rd.sim.run(name="ro_111622", components=['Q2', 'read_q2', 'TQ2', 'Store_Q2_2'], open_terminations=[], box_plus_buffer = False)
#eig_rd.sim.run(name="full_set2", components=['Q2', 'Store_Q2_2' 'read_q2', 'TQ2'], open_terminations=[], box_plus_buffer = False)

#eig_rd.sim.run(name="xmon_and_ro_set1", components=['Q1', 'Store_Q1_1', 'read_q1', 'TQ1'], open_terminations=[], box_plus_buffer = False)
#eig_rd.sim.run(name="storage_loss_fluxpad2", components=['Q2', 'Store_Q2_2', 'Launch_Flux2'], open_terminations=[], box_plus_buffer = False)
eig_rd.sim.plot_convergences()

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_arr

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

10:58PM 17s INFO [connect_meandered]: Zero meanders for Store_Q1_1
  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

10:58PM 17s INFO [connect_meandered]: Zero meanders for Store_Q2_2
  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

INFO 10:58PM [connect_project]: Connecting to Ansys Desktop API...
I

com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024349), None)

In [ ]:
eig_rd.sim.setup.n_modes =3

eig_rd.sim.setup.min_passes = 1
eig_rd.sim.setup.max_passes = 1

# eig_rd.sim.setup.min_passes = 15
# eig_rd.sim.setup.max_passes = 17


eig_rd.sim.setup_update(max_delta_f = 0.5, min_freq_ghz = 1.1)


design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q1'].options['hfss_inductance'] = 'Lj'
qcomps['Q1'].options['hfss_capacitance'] = 'Cj'

gui.rebuild()
gui.autoscale()


eig_rd.sim.run(name="chargecoupling", components=['Q1', 'Charge_Line_1'], open_terminations=[('Charge_Line_1', 'end')], box_plus_buffer = False)


eig_rd.sim.plot_convergences()

In [ ]:
eig_rd.sim.setup

Recover eigenmode frequencies for each variation.

Display the Ansys modeler window and plot the E-field on the chip's surface.

In [ ]:

hfss = eig_rd.sim.renderer
hfss.clear_fields(['Mag_E1'])
hfss.set_mode(1, "Setup")
hfss.modeler._modeler.ShowWindow()
hfss.plot_fields('main')
hfss.save_screenshot()

### EPR Analysis
Find the electric and magnetic energy stored in the readout system.

In [ ]:

eig_rd.sim.setup.name

In [20]:
# eig_rd.setup
# eig_rd.setup.junctions.jj.rect = 'JJ_rect_Lj_Q1_rect_jj'
# eig_rd.setup.junctions.jj.line = 'JJ_Lj_Q1_rect_jj_'

# eig_rd.setup

eig_rd.setup
eig_rd.setup.junctions.jj.rect = 'JJ_rect_Lj_Q2_rect_jj'
eig_rd.setup.junctions.jj.line = 'JJ_Lj_Q2_rect_jj_'

eig_rd.setup

{'junctions': {'jj': {'Lj_variable': 'Lj',
   'Cj_variable': 'Cj',
   'rect': 'JJ_rect_Lj_Q2_rect_jj',
   'line': 'JJ_Lj_Q2_rect_jj_'}},
 'dissipatives': {'dielectrics_bulk': ['main']},
 'cos_trunc': 8,
 'fock_trunc': 7,
 'sweep_variable': 'Lj'}

In [21]:

eig_rd.run_epr()

Design "storage_and_xmon_and_readout_final3_hfss" info:
	# eigenmodes    3
	# variations    1
Design "storage_and_xmon_and_readout_final3_hfss" info:
	# eigenmodes    3
	# variations    1


  options=pd.Series(get_instance_vars(self.options)),




        energy_elec_all       = 5.21029392355667e-24
        energy_elec_substrate = 4.78691247086828e-24
        EPR of substrate = 91.9%

        energy_mag    = 5.19827626174134e-24
        energy_mag % of energy_elec_all  = 99.8%
        

Variation 0  [1/1]


  Ljs = pd.Series({})

  Cjs = pd.Series({})

  _Om = pd.Series({})




  Mode 0 at 4.32 GHz   [1/3]
    Calculating ℰ_magnetic,ℰ_electric


...

###### Numerical diagonalization

Lj,7.5
0,4320.24
1,5330.49
2,6593.25


#### Kerr Non-linear coefficient table (MHz)

###### Numerical diagonalization

0       1         2
Lj                               
7.5 0  1.32e-03    1.13  1.14e-03
    1  1.13e+00  179.02  6.61e-01
    2  1.14e-03    0.66  7.37e-04

In [ ]:
eig_rd.sim.setup.vars.Lj 

In [ ]:
eig_rd.sim.close()

# LOM

In [ ]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

# example: update single setting
c1.sim.setup.max_passes = 10
# example: update multiple settings
c1.sim.setup_update(solution_order = 'Medium', auto_increase_solution_order = 'False')

c1.sim.setup


c1.sim.run(components=['Q1', 'Store_Q1_1','Storage_Line_1', 'Launch_Storage'], open_terminations=[('Storage_Line_1', 'start')])


In [ ]:
c1.sim.capacitance_matrix